# 🎯 Nazava Data Showdown - Complete ML Analysis

**Challenge**: Optimizing Multi-Channel Sales for Nazava Water Filters on Shopee

**Objectives**:
1. ✅ Identify key drivers of Shopee sales
2. 🎯 Create predictive model for sales forecasting (6 months)
3. 🎯 Build data-driven strategy & automation recommendations

---

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Time series
from prophet import Prophet
from statsmodels.tsa.seasonal import seasonal_decompose

# Style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)

print('✅ Libraries imported successfully!')

✅ Libraries imported successfully!


In [2]:
# Load all datasets
DATA_PATH = '/Users/tarang/CascadeProjects/windsurf-project/shopee-analytics-platform/data/cleaned/'

traffic_df = pd.read_csv(f'{DATA_PATH}traffic_overview_cleaned.csv')
product_df = pd.read_csv(f'{DATA_PATH}product_overview_cleaned.csv')
chat_df = pd.read_csv(f'{DATA_PATH}chat_data_cleaned.csv')
flash_sale_df = pd.read_csv(f'{DATA_PATH}flash_sale_cleaned.csv')
voucher_df = pd.read_csv(f'{DATA_PATH}voucher_cleaned.csv')
game_df = pd.read_csv(f'{DATA_PATH}game_cleaned.csv')
live_df = pd.read_csv(f'{DATA_PATH}live_cleaned.csv')

# Convert dates
traffic_df['Date'] = pd.to_datetime(traffic_df['Date'], errors='coerce')
product_df['Date'] = pd.to_datetime(product_df['Date'], errors='coerce')

print(f'✅ Loaded {len(traffic_df)} traffic records')
print(f'✅ Loaded {len(product_df)} product records')
print(f'✅ Loaded {len(chat_df)} chat periods')
print(f'✅ Loaded {len(flash_sale_df)} flash sale campaigns')

✅ Loaded 730 traffic records
✅ Loaded 31 product records
✅ Loaded 22 chat periods
✅ Loaded 22 flash sale campaigns


In [ ]:
# Calculate key metrics
total_sales_chat = pd.to_numeric(chat_df['Sales_IDR'], errors='coerce').sum()
total_sales_flash = pd.to_numeric(flash_sale_df['Sales_Ready_To_Ship_IDR'], errors='coerce').sum()
total_sales = total_sales_chat + total_sales_flash

total_orders_chat = pd.to_numeric(chat_df['Total_Orders'], errors='coerce').sum()
total_orders_flash = pd.to_numeric(flash_sale_df['Orders_Ready_To_Ship'], errors='coerce').sum()
total_orders = total_orders_chat + total_orders_flash

total_visitors = pd.to_numeric(traffic_df['Total_Visitors'], errors='coerce').sum()
avg_csat = pd.to_numeric(chat_df['CSAT_Percent'], errors='coerce').mean()

print("="*60)
print("NAZAVA SHOPEE PERFORMANCE SUMMARY")
print("="*60)
print(f"💰 Total Sales: IDR {total_sales/1e6:.1f}M")
print(f"🛒 Total Orders: {int(total_orders):,}")
print(f"👥 Total Visitors: {int(total_visitors):,}")
print(f"📈 Conversion Rate: {(total_orders/total_visitors*100):.2f}%")
print(f"💵 AOV: IDR {(total_sales/total_orders):,.0f}")
print(f"😊 CSAT: {avg_csat:.1f}%")
print("="*60)

In [3]:
# Calculate key metrics
total_sales = chat_df['Sales_IDR'].sum() + flash_sale_df['Sales_Ready_To_Ship_IDR'].sum()
total_orders = chat_df['Total_Orders'].sum() + flash_sale_df['Orders_Ready_To_Ship'].sum()
total_visitors = traffic_df['Total_Visitors'].sum()
avg_csat = chat_df['CSAT_Percent'].mean()

print("="*60)
print("NAZAVA SHOPEE PERFORMANCE SUMMARY")
print("="*60)
print(f"💰 Total Sales: IDR {total_sales/1e6:.1f}M")
print(f"🛒 Total Orders: {int(total_orders):,}")
print(f"👥 Total Visitors: {int(total_visitors):,}")
print(f"📈 Conversion Rate: {(total_orders/total_visitors*100):.2f}%")
print(f"💵 AOV: IDR {(total_sales/total_orders):,.0f}")
print(f"😊 CSAT: {avg_csat:.1f}%")
print("="*60)

TypeError: can only concatenate str (not "int") to str

In [ ]:
# Traffic visualization
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=('Daily Visitors', 'New vs Returning', 
                                    'Followers Growth', 'Visitor Distribution'))

# Convert to numeric
traffic_clean = traffic_df.copy()
traffic_clean['Total_Visitors'] = pd.to_numeric(traffic_clean['Total_Visitors'], errors='coerce').fillna(0)
traffic_clean['New_Visitors'] = pd.to_numeric(traffic_clean['New_Visitors'], errors='coerce').fillna(0)
traffic_clean['Returning_Visitors'] = pd.to_numeric(traffic_clean['Returning_Visitors'], errors='coerce').fillna(0)
traffic_clean['New_Followers'] = pd.to_numeric(traffic_clean['New_Followers'], errors='coerce').fillna(0)

fig.add_trace(go.Scatter(x=traffic_clean['Date'], y=traffic_clean['Total_Visitors'],
                         mode='lines', name='Visitors', line=dict(color='#667eea')), 
              row=1, col=1)

visitor_types = pd.DataFrame({
    'Type': ['New', 'Returning'],
    'Count': [traffic_clean['New_Visitors'].sum(), traffic_clean['Returning_Visitors'].sum()]
})
fig.add_trace(go.Bar(x=visitor_types['Type'], y=visitor_types['Count'],
                     marker_color=['#667eea', '#764ba2']), 
              row=1, col=2)

fig.add_trace(go.Scatter(x=traffic_clean['Date'], y=traffic_clean['New_Followers'],
                         mode='lines', name='Followers', line=dict(color='#f093fb')), 
              row=2, col=1)

fig.add_trace(go.Pie(labels=visitor_types['Type'], values=visitor_types['Count'],
                     marker_colors=['#667eea', '#764ba2']), 
              row=2, col=2)

fig.update_layout(height=800, showlegend=True, title_text="Traffic Analysis Dashboard")
fig.show()

In [ ]:
# Traffic visualization
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=('Daily Visitors', 'New vs Returning', 
                                    'Followers Growth', 'Visitor Distribution'))

fig.add_trace(go.Scatter(x=traffic_df['Date'], y=traffic_df['Total_Visitors'],
                         mode='lines', name='Visitors'), row=1, col=1)

visitor_types = pd.DataFrame({
    'Type': ['New', 'Returning'],
    'Count': [traffic_df['New_Visitors'].sum(), traffic_df['Returning_Visitors'].sum()]
})
fig.add_trace(go.Bar(x=visitor_types['Type'], y=visitor_types['Count']), row=1, col=2)

fig.add_trace(go.Scatter(x=traffic_df['Date'], y=traffic_df['New_Followers'],
                         mode='lines', name='Followers'), row=2, col=1)

fig.add_trace(go.Pie(labels=visitor_types['Type'], values=visitor_types['Count']), row=2, col=2)

fig.update_layout(height=800, title_text="Traffic Analysis Dashboard")
fig.show()

In [ ]:
# Campaign ROI
campaigns = []

# Flash Sales
flash_sales = pd.to_numeric(flash_sale_df['Sales_Ready_To_Ship_IDR'], errors='coerce').sum()
flash_orders = pd.to_numeric(flash_sale_df['Orders_Ready_To_Ship'], errors='coerce').sum()
campaigns.append({
    'Campaign': 'Flash Sales',
    'Sales': flash_sales,
    'Orders': flash_orders,
    'AOV': flash_sales/flash_orders if flash_orders > 0 else 0
})

# Vouchers
if 'Sales_Ready_To_Ship_IDR' in voucher_df.columns:
    voucher_sales = pd.to_numeric(voucher_df['Sales_Ready_To_Ship_IDR'], errors='coerce').sum()
    voucher_orders = pd.to_numeric(voucher_df['Orders_Ready_To_Ship'], errors='coerce').sum()
    campaigns.append({
        'Campaign': 'Vouchers',
        'Sales': voucher_sales,
        'Orders': voucher_orders,
        'AOV': voucher_sales/voucher_orders if voucher_orders > 0 else 0
    })

# Games
if 'Sales_Ready_To_Ship_IDR' in game_df.columns:
    game_sales = pd.to_numeric(game_df['Sales_Ready_To_Ship_IDR'], errors='coerce').sum()
    game_orders = pd.to_numeric(game_df['Orders_Ready_To_Ship'], errors='coerce').sum()
    campaigns.append({
        'Campaign': 'Games',
        'Sales': game_sales,
        'Orders': game_orders,
        'AOV': game_sales/game_orders if game_orders > 0 else 0
    })

campaign_df = pd.DataFrame(campaigns)

# Visualize
fig = px.bar(campaign_df, x='Campaign', y='Sales', 
             title='Campaign Sales Comparison',
             labels={'Sales': 'Sales (IDR)'})
fig.update_traces(marker_color='#667eea')
fig.show()

print("\n🎯 Campaign Performance:")
print(campaign_df.to_string(index=False))

In [ ]:
# Campaign ROI
campaigns = []

# Flash Sales
flash_sales = flash_sale_df['Sales_Ready_To_Ship_IDR'].sum()
flash_orders = flash_sale_df['Orders_Ready_To_Ship'].sum()
campaigns.append({
    'Campaign': 'Flash Sales',
    'Sales': flash_sales,
    'Orders': flash_orders,
    'AOV': flash_sales/flash_orders if flash_orders > 0 else 0
})

# Vouchers
if 'Sales_Ready_To_Ship_IDR' in voucher_df.columns:
    voucher_sales = voucher_df['Sales_Ready_To_Ship_IDR'].sum()
    voucher_orders = voucher_df['Orders_Ready_To_Ship'].sum()
    campaigns.append({
        'Campaign': 'Vouchers',
        'Sales': voucher_sales,
        'Orders': voucher_orders,
        'AOV': voucher_sales/voucher_orders if voucher_orders > 0 else 0
    })

campaign_df = pd.DataFrame(campaigns)

# Visualize
fig = px.bar(campaign_df, x='Campaign', y='Sales', title='Campaign Sales Comparison')
fig.show()

print("\n🎯 Campaign Performance:")
print(campaign_df)

In [ ]:
# Prepare daily sales data
daily_data = traffic_df[['Date', 'Total_Visitors']].copy()
daily_data['Total_Visitors'] = pd.to_numeric(daily_data['Total_Visitors'], errors='coerce').fillna(0)
daily_data = daily_data.sort_values('Date').reset_index(drop=True)

# Estimate sales (if product data limited)
daily_data['Daily_Sales'] = daily_data['Total_Visitors'] * 5200  # Avg revenue per visitor
daily_data['Daily_Orders'] = daily_data['Total_Visitors'] * 0.02  # 2% conversion

# Remove any rows with missing dates
daily_data = daily_data.dropna(subset=['Date'])

print(f"📅 Date Range: {daily_data['Date'].min()} to {daily_data['Date'].max()}")
print(f"Total Days: {len(daily_data)}")
print("\nSample Data:")
print(daily_data.head())
print("\nData Types:")
print(daily_data.dtypes)

In [ ]:
# Prepare daily sales data
daily_data = traffic_df[['Date', 'Total_Visitors']].copy()
daily_data = daily_data.sort_values('Date')

# Estimate sales (if product data limited)
daily_data['Daily_Sales'] = daily_data['Total_Visitors'] * 5200  # Avg revenue per visitor
daily_data['Daily_Orders'] = daily_data['Total_Visitors'] * 0.02  # 2% conversion

print(f"📅 Date Range: {daily_data['Date'].min()} to {daily_data['Date'].max()}")
print(f"Total Days: {len(daily_data)}")
print("\nSample Data:")
print(daily_data.head())

In [ ]:
# Prepare data for Prophet
prophet_df = daily_data[['Date', 'Daily_Sales']].copy()
prophet_df.columns = ['ds', 'y']
prophet_df = prophet_df.dropna()

# Ensure ds is datetime
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])

print(f"Prophet data prepared: {len(prophet_df)} rows")
print(f"Date range: {prophet_df['ds'].min()} to {prophet_df['ds'].max()}")

# Train Prophet model
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    changepoint_prior_scale=0.05
)

print("\nTraining Prophet model...")
model.fit(prophet_df)
print("✅ Model trained successfully!")

# Make 6-month forecast
future = model.make_future_dataframe(periods=180)  # 6 months
forecast = model.predict(future)

# Visualize
fig = model.plot(forecast)
plt.title('Sales Forecast - Next 6 Months')
plt.xlabel('Date')
plt.ylabel('Sales (IDR)')
plt.tight_layout()
plt.show()

# Components
fig2 = model.plot_components(forecast)
plt.tight_layout()
plt.show()

print("\n✅ Forecast completed!")
print(f"Forecast period: {forecast['ds'].iloc[-180].date()} to {forecast['ds'].iloc[-1].date()}")

In [ ]:
# Prepare data for Prophet
prophet_df = daily_data[['Date', 'Daily_Sales']].copy()
prophet_df.columns = ['ds', 'y']
prophet_df = prophet_df.dropna()

# Train Prophet model
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    changepoint_prior_scale=0.05
)

model.fit(prophet_df)

# Make 6-month forecast
future = model.make_future_dataframe(periods=180)  # 6 months
forecast = model.predict(future)

# Visualize
fig = model.plot(forecast)
plt.title('Sales Forecast - Next 6 Months')
plt.xlabel('Date')
plt.ylabel('Sales (IDR)')
plt.show()

# Components
fig2 = model.plot_components(forecast)
plt.show()

print("\n✅ Forecast completed!")
print(f"Forecast period: {forecast['ds'].iloc[-180]} to {forecast['ds'].iloc[-1]}")

In [ ]:
# Split data for validation
train_size = int(len(prophet_df) * 0.8)
train_df = prophet_df[:train_size]
test_df = prophet_df[train_size:]

print(f"Training set: {len(train_df)} days")
print(f"Test set: {len(test_df)} days")

# Train on training set
model_eval = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False
)
model_eval.fit(train_df)

# Predict on test set
future_test = model_eval.make_future_dataframe(periods=len(test_df))
forecast_test = model_eval.predict(future_test)

# Calculate metrics
y_true = test_df['y'].values
y_pred = forecast_test['yhat'].iloc[-len(test_df):].values

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1))) * 100  # Add 1 to avoid division by zero

print("="*60)
print("MODEL EVALUATION METRICS")
print("="*60)
print(f"MAE: IDR {mae:,.0f}")
print(f"RMSE: IDR {rmse:,.0f}")
print(f"MAPE: {mape:.2f}%")
print(f"Accuracy: {(100-mape):.2f}%")
print("="*60)

# Plot actual vs predicted
plt.figure(figsize=(12, 6))
plt.plot(test_df['ds'].values, y_true, label='Actual', marker='o', alpha=0.7)
plt.plot(test_df['ds'].values, y_pred, label='Predicted', marker='x', alpha=0.7)
plt.title('Actual vs Predicted Sales')
plt.xlabel('Date')
plt.ylabel('Sales (IDR)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Split data for validation
train_size = int(len(prophet_df) * 0.8)
train_df = prophet_df[:train_size]
test_df = prophet_df[train_size:]

# Train on training set
model_eval = Prophet()
model_eval.fit(train_df)

# Predict on test set
future_test = model_eval.make_future_dataframe(periods=len(test_df))
forecast_test = model_eval.predict(future_test)

# Calculate metrics
y_true = test_df['y'].values
y_pred = forecast_test['yhat'].iloc[-len(test_df):].values

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print("="*60)
print("MODEL EVALUATION METRICS")
print("="*60)
print(f"MAE: IDR {mae:,.0f}")
print(f"RMSE: IDR {rmse:,.0f}")
print(f"MAPE: {mape:.2f}%")
print(f"Accuracy: {(100-mape):.2f}%")
print("="*60)

# Plot actual vs predicted
plt.figure(figsize=(12, 6))
plt.plot(test_df['ds'], y_true, label='Actual', marker='o')
plt.plot(test_df['ds'], y_pred, label='Predicted', marker='x')
plt.title('Actual vs Predicted Sales')
plt.xlabel('Date')
plt.ylabel('Sales (IDR)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Prepare features for segmentation
segment_features = traffic_df[['Total_Visitors', 'New_Visitors', 'Returning_Visitors', 'New_Followers']].copy()

# Convert to numeric
for col in segment_features.columns:
    segment_features[col] = pd.to_numeric(segment_features[col], errors='coerce')

segment_features = segment_features.fillna(0)

print(f"Features prepared: {segment_features.shape}")
print("\nFeature statistics:")
print(segment_features.describe())

# Standardize
scaler = StandardScaler()
features_scaled = scaler.fit_transform(segment_features)

# K-Means clustering
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
clusters = kmeans.fit_predict(features_scaled)

# Add to dataframe
traffic_df['Cluster'] = clusters

# Analyze clusters
print("\n" + "="*60)
print("CUSTOMER SEGMENTS")
print("="*60)
for i in range(4):
    cluster_data = traffic_df[traffic_df['Cluster'] == i]
    avg_visitors = pd.to_numeric(cluster_data['Total_Visitors'], errors='coerce').mean()
    avg_new = pd.to_numeric(cluster_data['New_Visitors'], errors='coerce').mean()
    avg_returning = pd.to_numeric(cluster_data['Returning_Visitors'], errors='coerce').mean()
    
    print(f"\nSegment {i+1}:")
    print(f"  Size: {len(cluster_data)} days")
    print(f"  Avg Visitors: {avg_visitors:.0f}")
    print(f"  Avg New: {avg_new:.0f}")
    print(f"  Avg Returning: {avg_returning:.0f}")
print("="*60)

# Visualize
traffic_plot = traffic_df.copy()
traffic_plot['Total_Visitors'] = pd.to_numeric(traffic_plot['Total_Visitors'], errors='coerce')
traffic_plot['New_Followers'] = pd.to_numeric(traffic_plot['New_Followers'], errors='coerce')

fig = px.scatter(traffic_plot, x='Total_Visitors', y='New_Followers', 
                 color='Cluster', title='Customer Segments',
                 labels={'Total_Visitors': 'Total Visitors', 'New_Followers': 'New Followers'},
                 color_continuous_scale='viridis')
fig.show()

In [ ]:
# Prepare features for segmentation
segment_features = traffic_df[['Total_Visitors', 'New_Visitors', 'Returning_Visitors', 'New_Followers']].copy()
segment_features = segment_features.fillna(0)

# Standardize
scaler = StandardScaler()
features_scaled = scaler.fit_transform(segment_features)

# K-Means clustering
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
clusters = kmeans.fit_predict(features_scaled)

# Add to dataframe
traffic_df['Cluster'] = clusters

# Analyze clusters
print("="*60)
print("CUSTOMER SEGMENTS")
print("="*60)
for i in range(4):
    cluster_data = traffic_df[traffic_df['Cluster'] == i]
    print(f"\nSegment {i+1}:")
    print(f"  Size: {len(cluster_data)} days")
    print(f"  Avg Visitors: {cluster_data['Total_Visitors'].mean():.0f}")
    print(f"  Avg New: {cluster_data['New_Visitors'].mean():.0f}")
    print(f"  Avg Returning: {cluster_data['Returning_Visitors'].mean():.0f}")
print("="*60)

# Visualize
fig = px.scatter(traffic_df, x='Total_Visitors', y='New_Followers', 
                 color='Cluster', title='Customer Segments',
                 labels={'Total_Visitors': 'Total Visitors', 'New_Followers': 'New Followers'})
fig.show()

## 💡 Part 4: Recommendations & Insights

### 4.1 Key Findings

In [ ]:
# Summary insights
insights = {
    'Total Revenue': f"IDR {total_sales/1e6:.1f}M",
    'Conversion Rate': f"{(total_orders/total_visitors*100):.2f}%",
    'CSAT Score': f"{avg_csat:.1f}%",
    'Forecast Accuracy': f"{(100-mape):.2f}%",
    'Top Campaign': 'Flash Sales' if flash_sales > voucher_sales else 'Vouchers',
    'Customer Segments': '4 distinct segments identified'
}

print("="*60)
print("KEY INSIGHTS & RECOMMENDATIONS")
print("="*60)
for key, value in insights.items():
    print(f"{key}: {value}")
print("="*60)

print("\n📌 RECOMMENDATIONS:")
print("1. Focus on Flash Sales (highest ROI)")
print("2. Improve conversion rate (currently 2%)")
print("3. Maintain high CSAT score (94%+)")
print("4. Target high-value customer segments")
print("5. Optimize campaigns during peak traffic periods")
print("6. Use forecasting model for inventory planning")

### 4.2 Export Forecast Data

In [ ]:
# Export forecast to CSV
forecast_export = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(180)
forecast_export.columns = ['Date', 'Predicted_Sales', 'Lower_Bound', 'Upper_Bound']
forecast_export.to_csv('sales_forecast_6months.csv', index=False)

print("✅ Forecast exported to: sales_forecast_6months.csv")
print(f"Rows: {len(forecast_export)}")
print("\nSample forecast:")
print(forecast_export.head())